This is a tutorial for the DFODE-kit package.

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

from dfode_kit.df_interface import (
    OneDFreelyPropagatingFlameConfig,
    setup_one_d_flame_case,
    df_to_h5,
)
from dfode_kit.data_operations import (
    touch_h5, 
    get_TPY_from_h5, 
    random_perturb,
    label_npy,
    integrate_h5,
    calculate_error,
)
from dfode_kit.dfode_core.model.mlp import MLP
from dfode_kit.utils import BCT

DFODE_ROOT = os.environ['DFODE_ROOT']

print(DFODE_ROOT)

### A brief introduction to the DFODE method

#### Low-dimensional manifold sampling

A key challenge in preparing training data is achieving sufficient coverage of the relevant thermochemical composition space, which is often prohibitively high-dimensional when detailed chemistry involves tens to hundreds of species. 

To address this, DFODE-kit adopts a low-dimensional
manifold sampling strategy, where thermochemical states are extracted from canonical flame configurations that retain the essential topology of high-dimensional turbulent flames. This approach ensures both computational efficiency and physical representativeness of the training datasets.

In this tutorial, we will demonstrate how to use DFODE-kit to sample a low-dimensional manifold of thermochemical states from a one-dimensional laminar freely propagating flame simulated with DeepFlame. The following code block could also be found in `case_init.ipynb` files within the case templates provides in the `cases` directory. It is used to initialize the simulation and update the dictionary files for the simulation.

In [ ]:
# Operating condition settings
config_dict = {
    "mechanism": f"{DFODE_ROOT}/mechanisms/Burke2012_s9r23.yaml",
    "T0": 300,
    "p0": 101325,
    "fuel": "H2:1",
    "oxidizer": "O2:0.21,N2:0.79",
    "eq_ratio": 1.0,
}
config = OneDFreelyPropagatingFlameConfig(**config_dict)

# Simulation settings
settings = {
    "sim_time_step": 1e-6,
    "sim_write_interval": 1e-5,
    "num_output_steps": 10,
}
config.update_config(settings)

# Setup the case and update dictionary files
setup_one_d_flame_case(config, '.')

Note that at the point, the simulation is not yet started. The user would need to ensure a working version of DeepFlame is available and run the `Allrun` script from command line to start the simulation.

```bash
./Allrun
```

After the simulation is completed, we proceed to use DFODE-kit to gather and manage the thermochemical data.

In [ ]:
df_to_h5(
    root_dir=f"{DFODE_ROOT}/tutorials/oneD_freely_propagating_flame/1_sample_train",
    mechanism=f"{DFODE_ROOT}/mechanisms/Burke2012_s9r23.yaml",
    hdf5_file_path=f"{DFODE_ROOT}/tutorials/oneD_freely_propagating_flame/1_sample_train/tutorial_data.h5",
    include_mesh=True,
)

# The above is equivalent to the following cli command:
# dfode-kit sample --mech ../../mechanisms/Burke2012_s9r23.yaml \
#     --case . \
#     --save ./tutorial_data.h5 --include_mesh

Checking the contents of the h5 file

In [ ]:
touch_h5("tutorial_data.h5")

#### Data augmentation and labeling

While laminar canonical flames provide fundamental thermochemical states,their trajectory-aligned sampling in composition space poses significant limitations for a posteriori modeling applications. First, these sampled states are confined to predefined flamelet manifolds, making the trained model highly sensitive to perturbations and leading to an over-constrained representation. Second, the sampled states span a lower-dimensional subspace, which fails to encompass the full range of thermochemical variations encountered in turbulent combustion. As a result, the model becomes vulnerable to off-manifold perturbations—deviations from the training manifold that frequently arise in turbulent reacting flows.

To tackle this challenge, a data augmentation strategy is employed, where collected states are perturbed to simulate the effects of multi-dimensional transport and turbulence disturbances.

In [ ]:

h5_file = 'tutorial_data.h5'
mech = f'{DFODE_ROOT}/mechanisms/Burke2012_s9r23.yaml'
dataset_num = 20000
output_file = 'data'

print(f"Loading data from h5 file: {h5_file}")
data = get_TPY_from_h5(h5_file)    
print("Data shape:", data.shape)
All_data = random_perturb(data, mech, dataset_num, heat_limit=False, element_limit=True)
np.save(output_file, All_data)
print("Saved augmented data shape:", All_data.shape)
print(f"Saved augmented data to {output_file}")

# The above is equivalent to the following cli command:
# dfode-kit augment --mech ../../mechanisms/Burke2012_s9r23.yaml \
#     --h5_file ./tutorial_data.h5 \
#     --output_file ./data \
#     --dataset_num 20000

The CVODE integrator from Cantera is used for time integration and to provide supervised learning labels.

In [ ]:
from dfode_kit.data_operations import label_npy

try:
    labeled_data = label_npy(
        mech_path=mech,
        time_step=1e-06,
        source_path='./data.npy'
    )
    np.save('dataset.npy', labeled_data)
    print(f"Labeled data saved to: {'dataset.npy'}")
    
except (FileNotFoundError, ValueError) as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# The above is equivalent to the following cli command:
# dfode-kit label --mech ../../mechanisms/Burke2012_s9r23.yaml \
#     --time 1e-06 \
#     --source ./data.npy \
#     --save ./dataset.npy

#### Model training

Only a demo for training a model is provided here.

In [ ]:
import torch
import cantera as ct
from dfode_kit.dfode_core.train.formation import formation_calculate

source_file = 'dataset.npy'
time_step = 1e-06
output_path = './demo_model.pt'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
labeled_data = np.load(source_file)

gas = ct.Solution(mech)
n_species = gas.n_species
formation_enthalpies = formation_calculate(mech)

# Model instantiation
demo_model = MLP([2+n_species, 400, 400, 400, 400, n_species-1]).to(device)

# Data loading
thermochem_states1 = labeled_data[:, 0:2+n_species]
thermochem_states2 = labeled_data[:, 2+n_species:]

print(thermochem_states1.shape, thermochem_states2.shape)
thermochem_states1[:, 2:] = np.clip(thermochem_states1[:, 2:], 0, 1)
thermochem_states2[:, 2:] = np.clip(thermochem_states2[:, 2:], 0, 1)

features = torch.tensor(BCT(thermochem_states1), dtype=torch.float32).to(device)
labels = torch.tensor(BCT(thermochem_states2[:, 2:-1]) - BCT(thermochem_states1[:, 2:-1]), dtype=torch.float32).to(device)

features_mean = torch.mean(features, dim=0)
features_std = torch.std(features, dim=0)
features = (features - features_mean) / features_std

labels_mean = torch.mean(labels, dim=0)
labels_std = torch.std(labels, dim=0)
labels = (labels - labels_mean) / labels_std

formation_enthalpies = torch.tensor(formation_enthalpies, dtype=torch.float32).to(device)

# Training
loss_fn = torch.nn.L1Loss()
optimizer = torch.optim.Adam(demo_model.parameters(), lr=1e-3)

demo_model.train()  
for epoch in range(100):
    optimizer.zero_grad()
    preds = demo_model(features)
    loss1 = loss_fn(preds, labels)   ## LOSS  

    Y_in = ((features[:,2:-1]*features_std[2:-1] + features_mean[2:-1])*0.1 + 1)**10
    Y_out = (((preds*labels_std + labels_mean) + (features[:,2:-1]*features_std[2:-1] + features_mean[2:-1]))*0.1 + 1)**10
    Y_target = (((labels*labels_std + labels_mean) + (features[:,2:-1]*features_std[2:-1] + features_mean[2:-1]))*0.1 + 1)**10
    loss2 = loss_fn(Y_out.sum(axis=1), Y_in.sum(axis=1))

    Y_out_total = torch.cat((Y_out, (1-Y_out.sum(axis=1)).reshape(Y_out.shape[0],1)), axis = 1)
    Y_target_total = torch.cat((Y_target, (1-Y_target.sum(axis=1)).reshape(Y_target.shape[0],1)), axis = 1)
    loss3 = loss_fn((formation_enthalpies*Y_out_total).sum(axis=1), (formation_enthalpies*Y_target_total).sum(axis=1))/time_step

    loss = loss1 + loss2 + loss3/1e+13
    loss.backward()
    optimizer.step()
    
    print("Epoch: {}, Loss1: {:4e}, Loss2: {:4e}, Loss3: {:4e}, Loss: {:4e}".format(epoch+1, loss1.item(), loss2.item(), loss3.item(), loss.item()))

torch.save(
    {
        'net': demo_model.state_dict(),
        'data_in_mean': features_mean.cpu().numpy(),
        'data_in_std': features_std.cpu().numpy(),
        'data_target_mean': labels_mean.cpu().numpy(),
        'data_target_std': labels_std.cpu().numpy(),
    },
    output_path
)

# The above is equivalent to the following cli command:
# dfode-kit train --mech ../../mechanisms/Burke2012_s9r23.yaml     \
#     --source_file ./dataset.npy     \
#     --output_path ./demo_model.pt